<a href="https://colab.research.google.com/github/terkoizmy/Titanic/blob/main/Titanic_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [156]:
datatest = pd.read_csv('test.csv')
datatest

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [157]:
datatest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [158]:
datatest.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# Data Processing

In [159]:
# fill in the missing value in Age column with mean
datatest['Age'] = dataset['Age'].fillna(datatest['Age'].mean())

# delete column which we don't use
datatest.drop(columns = ["Cabin", "Ticket", "PassengerId", "Name", "Fare"], inplace = True)

In [160]:
datatest.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [161]:
Sex_encoder = LabelEncoder().fit(datatest['Sex'])
datatest['Sex'] = Sex_encoder.transform(datatest['Sex'])

Sex_encoder = LabelEncoder().fit(datatest['Embarked'])
datatest['Embarked'] = Sex_encoder.transform(datatest['Embarked'])

In [162]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
datatest = sc.fit_transform(datatest)

In [163]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
datatest = pca.fit_transform(datatest)

# Load Model

In [164]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [165]:
import xgboost as xgb
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.load_model("/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/XGBmodel.json")

In [166]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [167]:
clf = XGBClassifier()
booster = xgb.Booster()
booster.load_model('/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/XGBmodel.json')
clf._Booster = booster
clf._le = LabelEncoder().fit([0, 1])
XGBpredict = clf.predict(datatest)

In [168]:
len(XGBpredict)

418

In [169]:
subdat = pd.read_csv('gender_submission.csv')

In [170]:
subdat

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [171]:
subdat.drop(columns = ['Survived'], inplace = True)
subdat['Survived'] = XGBpredict

In [172]:
subdat

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [173]:
subdat.to_csv('/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/submit.csv',index=False)